# Generalizando Modelos Lineares

O objetivo de qualquer modelo de Machine Learning é generalizar para novoso conjuntos de dados. Não criamos o modelo para os dados de treino e sim para novos dados. Por isso é importante encontrar os melhores parâmertros para o modelo, que permitirão generalizar para novos dados. Existem diversas técnicas de regularização, de acordo com o modelo. Abaixo as 3 principais técnicas de regularização para modelos lineares: Ridge, LASSO e ElasticNet.

> http://scikit-learn.org/stable/modules/linear_model.html

In [1]:
# Import
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection  import train_test_split
from sklearn.model_selection  import cross_val_score, KFold, StratifiedKFold
#import sklearn.metrics
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Carga de dados e definição de variáveis  
boston = load_boston() 
dataset = pd.DataFrame(boston.data, columns = boston.feature_names)
dataset['target'] = boston.target

In [3]:
# Datasets
observations = len(dataset)
variables = dataset.columns[:-1]

In [4]:
# Variáveis X e Y
X = dataset.iloc[:,:-1]
y = dataset['target'].values

In [5]:
# Dados de treino e de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 101)

In [6]:
print ("Tamanho do Dataset de Treino: %i" % len(X_train))
print ("Tamanho do Dataset de Teste: %i" % len(X_test))

Tamanho do Dataset de Treino: 354
Tamanho do Dataset de Teste: 152


In [7]:
# Amostras de Treino, Validação e Teste
X_train, X_out_sample, y_train, y_out_sample = train_test_split(X, y, test_size = 0.40, random_state = 101)
X_validation, X_test, y_validation, y_test = train_test_split(X_out_sample, y_out_sample, test_size = 0.50, random_state = 101)

In [8]:
print ("Tamamnho da Amostra do Dataset de Treino: %i" % len(X_train))
print ("Tamamnho da Amostra do Dataset de Validação: %i" % len(X_validation))
print ("Tamamnho da Amostra do Dataset de Teste: %i" % len(X_test))

Tamamnho da Amostra do Dataset de Treino: 303
Tamamnho da Amostra do Dataset de Validação: 101
Tamamnho da Amostra do Dataset de Teste: 102


In [9]:
# Calcula o RMSE
def RMSE(y_true, y_pred):
    return np.sum((y_true - y_pred)**2)

In [10]:
# Cria o regressor linear
lm = LinearRegression()

In [11]:
# Divide as amostras em grupos
#cv_iterator = KFold(n = len(X), n_folds = 10, shuffle = True, random_state = 101)
cv_iterator = KFold(10, shuffle = True, random_state = 101)

In [12]:
# Define os limites do histograma
edges = np.histogram(y, bins = 5)[1]
edges

array([ 5., 14., 23., 32., 41., 50.])

In [13]:
# Retorna os índices dos compartimentos (bins) aos quais pertence cada valor na matriz de entrada.
binning = np.digitize(y, edges)
binning

array([3, 2, 4, 4, 4, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 1, 2,
       2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 3, 3, 4, 3, 3, 3,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 4, 3, 3, 3, 2, 2, 2, 2, 3, 4, 3,
       2, 2, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 3, 3, 3, 2, 3, 3, 2, 2,
       3, 3, 2, 2, 2, 3, 2, 3, 2, 4, 5, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 1, 5, 3, 3, 3, 6, 6, 6, 2, 3, 6, 3, 3, 2, 2, 2, 3, 3, 2, 3,
       3, 3, 3, 4, 4, 4, 4, 4, 3, 3, 6, 4, 3, 4, 4, 3, 4, 3, 3, 6, 4, 3,
       4, 4, 4, 3, 5, 5, 6, 2, 3, 2, 3, 2, 2, 2, 2, 3, 3, 3, 3, 3, 2, 3,
       3, 2, 3, 3, 5, 6, 4, 3, 5, 3, 3, 3, 5, 5, 3, 3, 3, 3, 3, 3, 2, 2,
       2, 3, 2, 2, 3, 2, 3, 3, 3, 3, 3, 5, 2, 2, 5, 6, 4, 3, 4, 5, 5, 3,
       4, 2, 3, 6, 5, 2, 2, 3, 3, 4, 4, 4, 4, 4, 3, 4, 5, 4, 5, 6, 4, 2,
       2, 3, 2, 3, 3, 4, 3, 3, 2, 3, 3, 2, 2, 3, 3,

In [14]:
# Divide as amostras em grupos com a mesma percentagem
#stratified_cv_iterator = StratifiedKFold(10, shuffle = True, random_state = 101)

Um padrão comum no aprendizado da máquina é usar modelos lineares treinados em funções não-lineares dos dados. Esta abordagem mantém o desempenho geralmente rápido dos métodos lineares, enquanto permite que eles se ajustem a uma gama muito maior de dados. Por exemplo, uma regressão linear simples pode ser estendida construindo características polinomiais a partir dos coeficientes. 

In [15]:
# Combinação polinomial de todos os atributos
# Gera uma nova matriz de características constituída por todas as combinações polinomiais 
# das características com grau menor ou igual ao grau especificado.
second_order = PolynomialFeatures(degree = 2, interaction_only = False)
second_order

PolynomialFeatures()

In [16]:
type(second_order)

sklearn.preprocessing._data.PolynomialFeatures

In [17]:
# Combinação polinomial de todos os atributos
third_order = PolynomialFeatures(degree = 3, interaction_only = True)
third_order

PolynomialFeatures(degree=3, interaction_only=True)

In [18]:
# Aplicando a combinação polinominal grau 2 aos atributos de entrada
over_param_X = second_order.fit_transform(X)
over_param_X

array([[1.00000000e+00, 6.32000000e-03, 1.80000000e+01, ...,
        1.57529610e+05, 1.97656200e+03, 2.48004000e+01],
       [1.00000000e+00, 2.73100000e-02, 0.00000000e+00, ...,
        1.57529610e+05, 3.62766600e+03, 8.35396000e+01],
       [1.00000000e+00, 2.72900000e-02, 0.00000000e+00, ...,
        1.54315409e+05, 1.58310490e+03, 1.62409000e+01],
       ...,
       [1.00000000e+00, 6.07600000e-02, 0.00000000e+00, ...,
        1.57529610e+05, 2.23851600e+03, 3.18096000e+01],
       [1.00000000e+00, 1.09590000e-01, 0.00000000e+00, ...,
        1.54802902e+05, 2.54955600e+03, 4.19904000e+01],
       [1.00000000e+00, 4.74100000e-02, 0.00000000e+00, ...,
        1.57529610e+05, 3.12757200e+03, 6.20944000e+01]])

In [19]:
# Aplicando a combinação polinominal grau 3 aos atributos de entrada
extra_over_param_X = third_order.fit_transform(X)
extra_over_param_X

array([[1.00000000e+00, 6.32000000e-03, 1.80000000e+01, ...,
        2.25534240e+04, 5.85062352e+05, 3.02413986e+04],
       [1.00000000e+00, 2.73100000e-02, 0.00000000e+00, ...,
        3.93714640e+04, 8.77895172e+05, 6.45724548e+04],
       [1.00000000e+00, 2.72900000e-02, 0.00000000e+00, ...,
        1.73596280e+04, 3.83111386e+05, 2.81792672e+04],
       ...,
       [1.00000000e+00, 6.07600000e-02, 0.00000000e+00, ...,
        3.23341200e+04, 6.11114868e+05, 4.70088360e+04],
       [1.00000000e+00, 1.09590000e-01, 0.00000000e+00, ...,
        3.71498400e+04, 6.96028788e+05, 5.35406760e+04],
       [1.00000000e+00, 4.74100000e-02, 0.00000000e+00, ...,
        4.51760400e+04, 8.53827156e+05, 6.56790120e+04]])

In [20]:
# Gerando o Score. Variáveis de entrada (X) com pré-processamento polinomial
cv_score = cross_val_score(lm, over_param_X, y, cv = cv_iterator, scoring = 'neg_mean_squared_error', n_jobs = 1)

In [21]:
print (cv_score)

[-11.67358612 -22.84201585  -9.19318785 -19.77458934 -11.68472903
  -9.15302456 -12.97982141 -22.18260706 -35.93064657 -13.75241158]


In [22]:
print ('Cv score: mean %0.3f std %0.3f' % (np.mean(np.abs(cv_score)), np.std(cv_score)))

Cv score: mean 16.917 std 7.955


In [23]:
#cv_score = cross_val_score(lm, over_param_X, y, cv = stratified_cv_iterator, scoring = 'neg_mean_squared_error', n_jobs = 1)

In [24]:
#print ('Cv score: mean %0.3f std %0.3f' % (np.mean(np.abs(cv_score)), np.std(cv_score)))

Métricas de Avaliação

http://scikit-learn.org/stable/modules/model_evaluation.html

## Regularização

De uma maneira bem direta, podemos entender regularização como a inserção de bias em um modelo. Ou em outras palavras, essa técnica desencoraja o ajuste excessivo dos dados, afim de diminuir a sua variância.

Dentro da regressão linear, Ridge e Lasso são formas de regularizarmos a nossa função através de penalidades. De forma simples, dentro de uma equação estatística dos dados, nós alteramos os fatores de forma a priorizar ou não certas parcelas da equação e, assim, evitamos ‘overfitting’ e melhoramos a qualidade de predição.

**Os parâmetros L1 e L2**<br/>
Uma regressão linear tenta ajustar uma função linear aos dados. O procedimento de ajuste envolve a função de custo como soma residual dos quadrados ou RSS. Os coeficientes w são escolhidos para minimizar essa função de custo com base nos dados de treinamento.<br/> 
 
No entanto, pode ocorrer overfitting, ou seja, o modelo pode “memorizar” o ruído dos dados de treinamento. Nesse caso, dizemos que o modelo tem um erro de generalização (erro na base de teste) elevado. Esse fenômeno está associado à variância do modelo, como vimos acima. Portanto, uma forma de diminuir o erro é aumentar o bias.<br/>
 
Para isso, regularizamos os coeficientes w, ou seja, restringimos o seu tamanho. Isso é feito adicionando um termo na função de custo, de forma que minimizar a função de custo automaticamente minimize também os coeficientes.

**Lasso (ou L1)**<br/>
Além de diminuir a variância do modelo, essa regularização tem uma outra importante aplicação em machine learning. Quando há múltiplas features altamente correlacionadas (ou seja, features que se comportam da mesma maneira) a regularização Lasso seleciona apenas uma dessas features e zera os coeficientes das outras, de forma a minimizar a penalização L1. Desse modo, dizemos que esse modelo realiza feature selection automaticamente, gerando vários coeficientes com peso zero, ou seja, que são ignorados pelo modelo. Isso facilita a interpretação do modelo, o que é uma enorme vantagem.

**Ridge (ou L2)**<br/>
Nesse caso, a penalização consiste nos quadrados dos coeficientes, ao invés de seus módulos. Qual será o efeito dessa regularização nos coeficientes de duas features altamente correlacionadas? Poderíamos ter duas features com coeficientes parecidos, ou uma com coeficiente alto, e outra com coeficiente zero. Como a penalização L2 é desproporcionalmente maior para coeficientes maiores, a regularização Ridge faz com que features correlacionadas tenham coeficientes parecidos.

No entanto, essa regularização não diminui a susceptibilidade do modelo a outliers, de forma que é recomendável limpar o dataset e remover features desnecessárias antes de realizar esse tipo de regressão.

Em termos matemáticos, a penalidade L1 não é diferenciável, o que pode complicar a sua implementação. Já a L2 é diferenciável, o que significa que ela pode ser usada em abordagens baseadas em gradiente.

**Elastic Net — L1+L2**<br/>
Sim, se trata exatamente de combinar os termos de regularização de L1 e L2. Assim, obtemos o melhor dos dois mundos, porém temos que enfrentar o problema de determinar dois hiperparâmetros para obter soluções ótimas.

**Referência:**<br/>
> DEVAY, Andre. *Modelos de Predição | Regressão de Ridge e Lasso*. Disponível em: https://medium.com/turing-talks/turing-talks-20-regressão-de-ridge-e-lasso-a0fc467b5629

### Ridge

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

In [25]:
from sklearn.linear_model import Ridge

In [26]:
# Cria o regressor Ridge
ridge = Ridge(normalize = True)

In [27]:
# Aplica o fit (Ridge) às variáveis
ridge.fit(second_order.fit_transform(X), y)

Ridge(normalize=True)

In [28]:
# Aplica o fit (Linear Regression) às variáveis
lm.fit(second_order.fit_transform(X), y)

LinearRegression()

In [29]:
# Comparação entre os coeficentes não regularizados (Linear Regression) e regularizados (Ridge)
# O objetivo da regularização é generalizar o modelo para novos conjuntos de dados
print ('Média dos Coeficientes: Não-regularizado = %0.3f Ridge = %0.3f' % (np.mean(lm.coef_), np.mean(ridge.coef_)))
print ('Coeficiente Mínimo: Não-regularizado = %0.3f Ridge = %0.3f' % (np.min(lm.coef_), np.min(ridge.coef_)))
print ('Max coefficient: Não-regularizado = %0.3f Ridge = %0.3f' % (np.max(lm.coef_), np.max(ridge.coef_)))

Média dos Coeficientes: Não-regularizado = 7592022.977 Ridge = -0.027
Coeficiente Mínimo: Não-regularizado = -35.002 Ridge = -2.011
Max coefficient: Não-regularizado = 797162270.345 Ridge = 1.181


### Lasso

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

In [30]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon

In [31]:
# Cria o regressor LASSO
lasso_reg = Lasso(alpha = 1.0, normalize = True, max_iter = 2*10**5)

In [32]:
# Random seed para garantir a reproducibilidade
np.random.seed(101)

In [33]:
# Divide as amostras em grupos com a mesma percentagem
cv_iterator = KFold(10, shuffle = True, random_state = 101)
#stratified_cv_iterator = StratifiedKFold(10, shuffle = True, random_state = 101)

In [34]:
# Busca pela melhor combinação de parâmetros testando diversos valores de alfa: {'alpha':np.logspace(-5,2,100)}
# A função RandomizedSearchCV gera o modelo como resultado
func_busca = RandomizedSearchCV(estimator = lasso_reg, 
                                param_distributions = {'alpha':np.logspace(-5,2,100)}, 
                                n_iter = 10, 
                                scoring='neg_mean_squared_error', 
                                n_jobs = 1, 
                                #iid = False, 
                                refit = True, 
                                cv = cv_iterator)

In [35]:
# Aplica o modelo criado aos datasets
# Esta operação leva algum tempo para ser executada.....
func_busca.fit(second_order.fit_transform(X), y)

RandomizedSearchCV(cv=KFold(n_splits=10, random_state=101, shuffle=True),
                   estimator=Lasso(max_iter=200000, normalize=True), n_jobs=1,
                   param_distributions={'alpha': array([1.00000000e-05, 1.17681195e-05, 1.38488637e-05, 1.62975083e-05,
       1.91791026e-05, 2.25701972e-05, 2.65608778e-05, 3.12571585e-05,
       3.67837977e-05, 4.32876128e-05, 5.09413801e-05, 5.99484250e-05,
       7.0548...
       2.36448941e+00, 2.78255940e+00, 3.27454916e+00, 3.85352859e+00,
       4.53487851e+00, 5.33669923e+00, 6.28029144e+00, 7.39072203e+00,
       8.69749003e+00, 1.02353102e+01, 1.20450354e+01, 1.41747416e+01,
       1.66810054e+01, 1.96304065e+01, 2.31012970e+01, 2.71858824e+01,
       3.19926714e+01, 3.76493581e+01, 4.43062146e+01, 5.21400829e+01,
       6.13590727e+01, 7.22080902e+01, 8.49753436e+01, 1.00000000e+02])},
                   scoring='neg_mean_squared_error')

In [36]:
# Imprime os resultados
print ('Melhor valor de alpha: %0.5f' % func_busca.best_params_['alpha'])
print ('Melhor CV mean squared error: %0.3f' % np.abs(func_busca.best_score_))

Melhor valor de alpha: 0.00014
Melhor CV mean squared error: 13.357


In [37]:
print ('Coeficients com valor igual a zero: %i de %i' % (np.sum(~(func_busca.best_estimator_.coef_ == 0.0)), len(func_busca.best_estimator_.coef_)))

Coeficients com valor igual a zero: 76 de 105


In [38]:
# Aplica a regularização Lasso com Cross Validation (o melhor modelo é selecionado por Cross Validation)
lasso_reg_cv = LassoCV(alphas = np.logspace(-5,2,100), normalize = True, n_jobs = 1, cv = None, max_iter = 10**6)

In [39]:
# Fit
# Esta operação leva algum tempo para ser executada.....
lasso_reg_cv.fit(second_order.fit_transform(X), y)

LassoCV(alphas=array([1.00000000e-05, 1.17681195e-05, 1.38488637e-05, 1.62975083e-05,
       1.91791026e-05, 2.25701972e-05, 2.65608778e-05, 3.12571585e-05,
       3.67837977e-05, 4.32876128e-05, 5.09413801e-05, 5.99484250e-05,
       7.05480231e-05, 8.30217568e-05, 9.77009957e-05, 1.14975700e-04,
       1.35304777e-04, 1.59228279e-04, 1.87381742e-04, 2.20513074e-04,
       2.59502421e-04, 3.05385551e-0...
       2.36448941e+00, 2.78255940e+00, 3.27454916e+00, 3.85352859e+00,
       4.53487851e+00, 5.33669923e+00, 6.28029144e+00, 7.39072203e+00,
       8.69749003e+00, 1.02353102e+01, 1.20450354e+01, 1.41747416e+01,
       1.66810054e+01, 1.96304065e+01, 2.31012970e+01, 2.71858824e+01,
       3.19926714e+01, 3.76493581e+01, 4.43062146e+01, 5.21400829e+01,
       6.13590727e+01, 7.22080902e+01, 8.49753436e+01, 1.00000000e+02]),
        max_iter=1000000, n_jobs=1, normalize=True)

In [40]:
# Imprime os resultados
print ('Melhor valor de alpha: %0.5f' % lasso_reg_cv.alpha_)

Melhor valor de alpha: 0.00413


### Elasticnet

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html

In [41]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon

In [42]:
# Cria o regressor ElasrticNet
elasticnet_reg = ElasticNet(alpha = 1.0, l1_ratio = 0.15, normalize = True, max_iter = 10**6, random_state = 101)

In [43]:
np.random.seed(101)

In [44]:
# Busca pela melhor combinação de parâmetros testando diversos valores de alfa: {'alpha':np.logspace(-5,2,100)}
# A função RandomizedSearchCV gera o modelo como resultado
func_busca = RandomizedSearchCV(estimator = elasticnet_reg, 
                                param_distributions = {'alpha':np.logspace(-5,2,100), 
                                                       'l1_ratio':np.arange(0.0, 1.01, 0.05)}, 
                                n_iter = 10, 
                                scoring = 'neg_mean_squared_error', 
                                n_jobs = 1, 
                                #iid = False, 
                                refit = True, 
                                cv = cv_iterator)

In [45]:
# Fit
func_busca.fit(second_order.fit_transform(X), y)

RandomizedSearchCV(cv=KFold(n_splits=10, random_state=101, shuffle=True),
                   estimator=ElasticNet(l1_ratio=0.15, max_iter=1000000,
                                        normalize=True, random_state=101),
                   n_jobs=1,
                   param_distributions={'alpha': array([1.00000000e-05, 1.17681195e-05, 1.38488637e-05, 1.62975083e-05,
       1.91791026e-05, 2.25701972e-05, 2.65608778e-05, 3.12571585e-05,
       3.67837977e-05, 4.32876128e-05...
       8.69749003e+00, 1.02353102e+01, 1.20450354e+01, 1.41747416e+01,
       1.66810054e+01, 1.96304065e+01, 2.31012970e+01, 2.71858824e+01,
       3.19926714e+01, 3.76493581e+01, 4.43062146e+01, 5.21400829e+01,
       6.13590727e+01, 7.22080902e+01, 8.49753436e+01, 1.00000000e+02]),
                                        'l1_ratio': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])},
                   scoring='neg_mean

In [46]:
# Resultados
print ('Melhor valor de alpha: %0.5f' % func_busca.best_params_['alpha'])
print ('Melhor L1_ratio: %0.5f' % func_busca.best_params_['l1_ratio'])
print ('Melhor CV mean squared error: %0.3f' % np.abs(func_busca.best_score_))

Melhor valor de alpha: 0.00002
Melhor L1_ratio: 0.60000
Melhor CV mean squared error: 12.987


In [47]:
# Coeficientes 
print ('Coeficients com valor igual a zero: %i de %i' % (np.sum(~(func_busca.best_estimator_.coef_ == 0.0)), 
                                                         len(func_busca.best_estimator_.coef_)))

Coeficients com valor igual a zero: 102 de 105


In [48]:
# Aplica a regularização ElasticNet com Cross Validation (o melhor modelo é selecionado por Cross Validation)
elasticnet_reg_cv = ElasticNetCV(alphas = np.logspace(-5,2,100), 
                                 normalize = True, 
                                 n_jobs = 1, 
                                 cv = None, 
                                 max_iter = 10**6)

In [49]:
# Fit
elasticnet_reg_cv.fit(second_order.fit_transform(X), y)

ElasticNetCV(alphas=array([1.00000000e-05, 1.17681195e-05, 1.38488637e-05, 1.62975083e-05,
       1.91791026e-05, 2.25701972e-05, 2.65608778e-05, 3.12571585e-05,
       3.67837977e-05, 4.32876128e-05, 5.09413801e-05, 5.99484250e-05,
       7.05480231e-05, 8.30217568e-05, 9.77009957e-05, 1.14975700e-04,
       1.35304777e-04, 1.59228279e-04, 1.87381742e-04, 2.20513074e-04,
       2.59502421e-04, 3.053855...
       2.36448941e+00, 2.78255940e+00, 3.27454916e+00, 3.85352859e+00,
       4.53487851e+00, 5.33669923e+00, 6.28029144e+00, 7.39072203e+00,
       8.69749003e+00, 1.02353102e+01, 1.20450354e+01, 1.41747416e+01,
       1.66810054e+01, 1.96304065e+01, 2.31012970e+01, 2.71858824e+01,
       3.19926714e+01, 3.76493581e+01, 4.43062146e+01, 5.21400829e+01,
       6.13590727e+01, 7.22080902e+01, 8.49753436e+01, 1.00000000e+02]),
             max_iter=1000000, n_jobs=1, normalize=True)

In [50]:
# Resultados
print ('Melhor valor de alpha: %0.5f' % elasticnet_reg_cv.alpha_)
print ('Melhor L1_ratio: %0.5f' % elasticnet_reg_cv.l1_ratio_)

Melhor valor de alpha: 0.00132
Melhor L1_ratio: 0.50000
